In [333]:
#!/usr/bin/env python
from googleads import adwords

In [323]:
import time

In [324]:
from StringIO import StringIO
import pandas as pd
import numpy as np

In [374]:
class pause_bad_performing_ads(object):
    def __init__(self, country, start, end, max_ads=6):
        self._country = country
        self._client = adwords.AdWordsClient.LoadFromStorage()
        self._page_size = 100
        self._max_ads = max_ads
        self._start = start
        self._end = end
        self._active = 20170718
        
    
    def get_accounts(self):
        managed_customer_service = self._client.GetService('ManagedCustomerService', version='v201609')
        selector = {
            'fields': ['CustomerId', 'Name']
        }    
        account_ids = []
        page = managed_customer_service.get(selector)
        if 'entries' in page and page['entries']:
            for account in page['entries']:
                if str(account['name']).startswith(self._country + '_') and 'jobseeker' in str(account['name']) \
                    and '_aed_' not in str(account['name']) and '_app' not in str(account['name'])\
                    and '_display_' not in str(account['name']) and '_content_' not in str(account['name']):              
                    account_ids.append(str(account['customerId']))         
        return account_ids
    
    def get_campaigns(self, account_id):
        self._client.SetClientCustomerId(account_id)
        campaign_service = self._client.GetService('CampaignService', version='v201609')
    
        enabled_campaigns = []
        offset = 0
        qry = 'SELECT Id, Name, Status ORDER BY Name LIMIT {lower}, {upper}'
    
        more_pages = True
        while more_pages:
            page = campaign_service.query(qry.format(lower=offset, upper=self._page_size))
    
            if 'entries' in page:
                for campaign in page['entries']:
                    if campaign['status']=='ENABLED' and campaign['name'].lower().find("dynamic search ads")==-1:
                        enabled_campaigns.append(str(campaign['id']))
    
            offset += self._page_size
            more_pages = offset < int(page['totalNumEntries'])
            time.sleep(1)
        return enabled_campaigns
    
    
    def get_ad_groups(self, account_id, campaign_id):
        self._client.SetClientCustomerId(account_id)
        ad_group_service = self._client.GetService('AdGroupService', version='v201609')
        ad_groups = []
    
        offset = 0
        query = "SELECT Id, BaseAdGroupId, Name, Status WHERE CampaignId= '{0}'".format(campaign_id)
        more_pages = True
        while more_pages:
            page = ad_group_service.query(query + ' LIMIT %s, %s' % (offset, self._page_size))
            if 'entries' in page:
                for ad_group in page['entries']:
                    if ad_group['status'] == 'ENABLED':
                        ad_groups.append(str(ad_group['id']))
            offset += self._page_size
            more_pages = offset < int(page['totalNumEntries'])
        return ad_groups


    def get_ad_data(self, account_id, campaign_id):
        self._client.SetClientCustomerId(account_id)
        report_downloader = self._client.GetReportDownloader(version='v201705')
        qry = ('SELECT CampaignId, AdGroupId, Id, Impressions, Clicks, Cost FROM '
               'AD_PERFORMANCE_REPORT WHERE Status IN [ENABLED] AND CampaignId={2} '
               'DURING {0}, {1}').format(self._start, self._end, campaign_id)  
        stream_data = report_downloader.DownloadReportAsStringWithAwql(qry, 'CSV',
                skip_report_header=True, skip_column_header=False,
                skip_report_summary=True, include_zero_impressions=True)
        report_data = StringIO(stream_data)
        report_df = pd.DataFrame.from_csv(report_data, sep=',').reset_index()
        return report_df
    
    def low_impression_share(self, account_id, campaign_id):
        data = self.get_ad_data(account_id, campaign_id)
        count_ads = data.groupby(['Campaign ID', 'Ad group ID'])['Ad ID'].count().reset_index()
        sum_impressions = data.groupby(['Campaign ID', 'Ad group ID'])['Impressions'].sum().reset_index()   
        sum_impressions.columns = ['Campaign ID', 'Ad group ID', 'Total Impressions']
        #adgroup with greater than _max_ads
        filtered = count_ads[count_ads['Ad ID']>self._max_ads][['Campaign ID', 'Ad group ID']]
        #add info 
        with_info = pd.merge(filtered, data, how='left', on=['Campaign ID', 'Ad group ID'])
        with_info = pd.merge(with_info, sum_impressions, how='left', on=['Campaign ID', 'Ad group ID'])
        with_info = with_info[['Ad group ID', 'Ad ID', 'Impressions', 'Clicks', 'Total Impressions']]
        #ctr & impression share
        with_info['ctr'] = with_info['Clicks']/with_info['Impressions']
        with_info['imp_share'] = with_info['Impressions']/with_info['Total Impressions']
        with_info = with_info.fillna(0)
        #rank
        share_rank = with_info.groupby(['Ad group ID'])['imp_share'].rank(ascending=False).to_frame()
        share_rank.columns = ['share_rank']
        #merged
        merged = pd.merge(with_info, share_rank, left_index=True, right_index=True)
        merged = merged.sort_values(['Ad group ID', 'share_rank'])
        return merged


    def ads_to_pause_low_ctr_high_std_dev(self, account_id, campaign_id):
        data = self.get_ad_data(account_id, campaign_id)
        count_ads = data.groupby(['Campaign ID', 'Ad group ID'])['Ad ID'].count().reset_index()
        #adgroup with greater than _max_ads
        filtered = count_ads[count_ads['Ad ID']>self._max_ads][['Campaign ID', 'Ad group ID']]
        #add info 
        with_info = pd.merge(filtered, data, how='left', on=['Campaign ID', 'Ad group ID'])
        with_info = with_info[['Ad group ID', 'Ad ID', 'Impressions', 'Clicks']]
        #ctr
        with_info['ctr'] = with_info['Clicks']/with_info['Impressions']
        with_info = with_info.fillna(0)
        with_info['std'] = np.sqrt((with_info['ctr']*(1-with_info['ctr']))/with_info['Impressions'])
        with_info = with_info.fillna(0)
        with_info['ctr_adjusted'] = with_info['ctr'] - with_info['std']
        #rank
        ranking = with_info.groupby(['Ad group ID'])['ctr_adjusted'].rank(ascending=False).to_frame()
        ranking.columns = ['ranking']
        #merged
        merged = pd.merge(with_info, ranking, left_index=True, right_index=True)
        #rank >= max_ads
        #TODO: handle ties
        to_be_paused = merged[(merged.ranking>=self._max_ads)]
        return zip(list(to_be_paused['Ad group ID'].values),list(to_be_paused['Ad ID'].values))   

        
    def ads_to_pause(self, account_id, campaign_id):
        data = self.get_ad_data(account_id, campaign_id)
        count_ads = data.groupby(['Campaign ID', 'Ad group ID'])['Ad ID'].count().reset_index()
        #adgroup with greater than _max_ads
        filtered = count_ads[count_ads['Ad ID']>self._max_ads][['Campaign ID', 'Ad group ID']]
        #add info 
        with_info = pd.merge(filtered, data, how='left', on=['Campaign ID', 'Ad group ID'])
        with_info = with_info[['Ad group ID', 'Ad ID', 'Impressions', 'Clicks']]
        #ctr
        with_info['ctr'] = with_info['Clicks']/with_info['Impressions']
        with_info = with_info.fillna(0)
        #rank
        ranking = with_info.groupby(['Ad group ID'])['ctr'].rank(ascending=False).to_frame()
        ranking.columns = ['ranking']
        #merged
        merged = pd.merge(with_info, ranking, left_index=True, right_index=True)
        #rank >= max_ads & impressions > 100
        to_be_paused = merged[(merged.ranking>=self._max_ads)&(merged.Impressions>100)]
        return zip(list(to_be_paused['Ad group ID'].values),list(to_be_paused['Ad ID'].values))
    

    def pause_ads(self, account_id, ad_group_id, ad_id, verbose=True):
        self._client.SetClientCustomerId(account_id)
        ad_group_ad_service = self._client.GetService('AdGroupAdService', version='v201705')
    
        operations = [{
            'operator': 'SET',
            'operand': {
                'adGroupId': ad_group_id,
                'ad': {'id': ad_id},
                'status': 'PAUSED'
            }
        }]
        ads = ad_group_ad_service.mutate(operations)
        
        if verbose:
            # Display results.
            for ad in ads['value']:
                print 'Ad with id \'%s\' was updated.'% ad['ad']['id']    
    
    def run(self):
        accounts = self.get_accounts()
        for account in accounts:
            campaigns = self.get_campaigns(account)
            for campaign in campaigns:
                pause_these = self.ads_to_pause(account, campaign)
                if pause_these:
                    for k in pause_these:
                        ad_group, ad = k
                        self.pause_ads(account, ad_group, ad)
                        
    def pause_low_ctr_high_std_dev(self):
        accounts = self.get_accounts()
        for account in accounts:
            campaigns = self.get_campaigns(account)
            for campaign in campaigns:
                pause_these = self.ads_to_pause_low_ctr_high_std_dev(account, campaign)
                print pause_these
                if pause_these:
                    for k in pause_these:
                        ad_group, ad = k
                        try:
                            self.pause_ads(account, ad_group, ad)
                        except:
                            pass

In [375]:
a = pause_bad_performing_ads('fi', '20170101', '20170731')

In [372]:
z = a.ads_to_pause_low_ctr_high_std_dev('7473408527', '147796423').sort_values(['Ad group ID', 'ranking'])

AttributeError: 'list' object has no attribute 'sort_values'

In [367]:
z[z['Ad group ID']==7923636103]

,Ad group ID,Ad ID,Impressions,Clicks,ctr,std,ctr_adjusted,ranking
4295,7923636103,163981854156,17674,2479,0.140263,0.002612,0.137650,1
4297,7923636103,163981854141,5366,656,0.122251,0.004472,0.117779,2
4292,7923636103,163981854138,4208,489,0.116207,0.004940,0.111267,3
4289,7923636103,37709698903,2425,274,0.112990,0.006429,0.106561,4
4294,7923636103,163981854150,1302,150,0.115207,0.008848,0.106359,5
4299,7923636103,163981854159,3009,337,0.111997,0.005749,0.106248,6
4291,7923636103,37709699023,293,35,0.119454,0.018947,0.100507,7
4298,7923636103,163981854144,960,95,0.098958,0.009637,0.089321,8
4296,7923636103,163981854153,947,90,0.095037,0.009530,0.085507,9
4300,7923636103,163981854162,5174,378,0.073058,0.003618,0.069440,10


In [376]:
a.pause_low_ctr_high_std_dev()

[(7923691543, 37709859223), (7923691543, 163980802232), (7923691543, 163980802241), (7923691543, 163980802244), (7923691543, 163980802247), (7923691543, 37709858983), (7923691543, 37709859103), (7923691543, 163980802226), (7923691543, 163980802229), (7923691543, 163980802235), (7923691543, 163980802238), (7923691543, 163980802250), (7923691543, 163980802253), (7923691663, 37709859463), (7923691663, 163980808985), (7923691663, 163980808988), (7923691663, 37709859583), (7923691663, 163980808976), (7923691663, 163980808982), (7923691663, 163980808994), (7923691663, 163980808997), (7923691663, 163980809000), (7923691903, 37709859943), (7923691903, 37709860063), (7923691903, 163980803423), (7923691903, 163980803432), (7923691903, 163980803435), (7923691903, 163980803438), (7923691903, 163980803444), (7923691903, 163980803447), (7923692143, 37709860303), (7923692143, 37709860423), (7923692143, 37709860543), (7923692143, 163980013083), (7923692143, 163980013089), (7923692143, 163980013095), (

Ad with id '37709859463' was updated.
Ad with id '163980808985' was updated.
Ad with id '163980808988' was updated.
Ad with id '37709859583' was updated.
Ad with id '163980808976' was updated.
Ad with id '163980808982' was updated.
Ad with id '163980808994' was updated.
Ad with id '163980808997' was updated.
Ad with id '163980809000' was updated.
Ad with id '37709859943' was updated.
Ad with id '37709860063' was updated.
Ad with id '163980803423' was updated.
Ad with id '163980803432' was updated.
Ad with id '163980803435' was updated.
Ad with id '163980803438' was updated.
Ad with id '163980803444' was updated.
Ad with id '163980803447' was updated.
Ad with id '37709860303' was updated.
Ad with id '37709860423' was updated.
Ad with id '37709860543' was updated.
Ad with id '163980013083' was updated.
Ad with id '163980013089' was updated.
Ad with id '163980013095' was updated.
Ad with id '163980013098' was updated.
Ad with id '163980013110' was updated.
Ad with id '163980013086' was up

Ad with id '163980154761' was updated.
Ad with id '163975464370' was updated.
Ad with id '163975464382' was updated.
Ad with id '37709868823' was updated.
Ad with id '37709869063' was updated.
Ad with id '163975464367' was updated.
Ad with id '163975464373' was updated.
Ad with id '163975464379' was updated.
Ad with id '163975464385' was updated.
Ad with id '163975464388' was updated.
Ad with id '163975464391' was updated.
Ad with id '37709869423' was updated.
Ad with id '163980017730' was updated.
Ad with id '163980017733' was updated.
Ad with id '163980017739' was updated.
Ad with id '163980017751' was updated.
Ad with id '163980017754' was updated.
Ad with id '37709869303' was updated.
Ad with id '163980017727' was updated.
Ad with id '163980017736' was updated.
Ad with id '163980017742' was updated.
Ad with id '163980017745' was updated.
Ad with id '37709869543' was updated.
Ad with id '37709869663' was updated.
Ad with id '163980152061' was updated.
Ad with id '163980152064' was u

Ad with id '163980149745' was updated.
Ad with id '163980149748' was updated.
Ad with id '163980149751' was updated.
Ad with id '163980149754' was updated.
Ad with id '163980149757' was updated.
Ad with id '37709877703' was updated.
Ad with id '163980149739' was updated.
Ad with id '163980149742' was updated.
Ad with id '163980149880' was updated.
Ad with id '163980149883' was updated.
Ad with id '37709877943' was updated.
Ad with id '37709878183' was updated.
Ad with id '163975463227' was updated.
Ad with id '163975463233' was updated.
Ad with id '163975463242' was updated.
Ad with id '163975463245' was updated.
Ad with id '163975463251' was updated.
Ad with id '37709878063' was updated.
Ad with id '163975463230' was updated.
Ad with id '163975463236' was updated.
Ad with id '163975463239' was updated.
Ad with id '163980013377' was updated.
Ad with id '163980013383' was updated.
Ad with id '163980013386' was updated.
Ad with id '163980013398' was updated.
Ad with id '37709878423' was 

Ad with id '163980012465' was updated.
Ad with id '163980012471' was updated.
Ad with id '37709886343' was updated.
Ad with id '163980927482' was updated.
Ad with id '163980927488' was updated.
Ad with id '37709886463' was updated.
Ad with id '37709886583' was updated.
Ad with id '163980927353' was updated.
Ad with id '163980927485' was updated.
Ad with id '163980927497' was updated.
Ad with id '37709887063' was updated.
Ad with id '163980929423' was updated.
Ad with id '163980929441' was updated.
Ad with id '37709886823' was updated.
Ad with id '37709886943' was updated.
Ad with id '163980929417' was updated.
Ad with id '163980929420' was updated.
Ad with id '163980929426' was updated.
Ad with id '163980929429' was updated.
Ad with id '163980929432' was updated.
Ad with id '163980929435' was updated.
Ad with id '163980929438' was updated.
Ad with id '37709887663' was updated.
Ad with id '37709887783' was updated.
Ad with id '163980806282' was updated.
Ad with id '163980806291' was upd

...

Ad with id '163981748357' was updated.
Ad with id '163981748366' was updated.
Ad with id '163981748369' was updated.
Ad with id '163981748375' was updated.
Ad with id '163981748381' was updated.
Ad with id '37710239263' was updated.
Ad with id '37710239023' was updated.
Ad with id '37710239143' was updated.
Ad with id '163981747382' was updated.
Ad with id '163981747397' was updated.
Ad with id '163981747370' was updated.
Ad with id '163981747373' was updated.
Ad with id '163981747376' was updated.
Ad with id '163981747385' was updated.
Ad with id '163981747388' was updated.
Ad with id '163981747391' was updated.
Ad with id '37710240463' was updated.
Ad with id '37710240583' was updated.
Ad with id '37710240703' was updated.
Ad with id '163981747832' was updated.
Ad with id '163981747841' was updated.
Ad with id '163981747844' was updated.
Ad with id '163981747847' was updated.
Ad with id '163981747835' was updated.
Ad with id '163981747850' was updated.
Ad with id '163981747853' was u

Ad with id '37710256663' was updated.
Ad with id '37710256423' was updated.
Ad with id '37710256543' was updated.
Ad with id '37710258343' was updated.
Ad with id '37710258463' was updated.
Ad with id '163981204941' was updated.
Ad with id '163981204947' was updated.
Ad with id '163981204950' was updated.
Ad with id '163981204959' was updated.
Ad with id '163981204965' was updated.
Ad with id '163981204938' was updated.
Ad with id '163981204944' was updated.
Ad with id '163981204953' was updated.
Ad with id '163981204956' was updated.
Ad with id '163981204962' was updated.
Ad with id '163981742624' was updated.
Ad with id '163981742630' was updated.
Ad with id '163981742633' was updated.
Ad with id '163981742765' was updated.
Ad with id '163981742768' was updated.
Ad with id '163981742627' was updated.
Ad with id '163981742636' was updated.
Ad with id '163981742762' was updated.
Ad with id '37710258703' was updated.
Ad with id '37710258823' was updated.
Ad with id '37710259063' was upd

Ad with id '163981746689' was updated.
Ad with id '163981746692' was updated.
Ad with id '163981746695' was updated.
Ad with id '163981746707' was updated.
Ad with id '163981746710' was updated.
Ad with id '37710278503' was updated.
Ad with id '37710278623' was updated.
Ad with id '37710278383' was updated.
Ad with id '37710280663' was updated.
Ad with id '37710280423' was updated.
Ad with id '37710280543' was updated.
Ad with id '163981179285' was updated.
Ad with id '163981179288' was updated.
Ad with id '163981179291' was updated.
Ad with id '163981179294' was updated.
Ad with id '163981179309' was updated.
Ad with id '163981179297' was updated.
Ad with id '163981179300' was updated.
Ad with id '163981179303' was updated.
Ad with id '163981179306' was updated.
Ad with id '163981179312' was updated.
Ad with id '37710280783' was updated.
Ad with id '37710280903' was updated.
Ad with id '37710281023' was updated.
Ad with id '163981196133' was updated.
Ad with id '163981196139' was upda

Ad with id '37710294463' was updated.
Ad with id '37710294223' was updated.
Ad with id '37710296263' was updated.
Ad with id '163981192488' was updated.
Ad with id '163981192491' was updated.
Ad with id '163981192500' was updated.
Ad with id '163981192506' was updated.
Ad with id '37710296143' was updated.
Ad with id '163981192485' was updated.
Ad with id '163981192494' was updated.
Ad with id '163981192497' was updated.
Ad with id '163981192503' was updated.
Ad with id '163981192509' was updated.
Ad with id '163981192512' was updated.
Ad with id '163981174965' was updated.
Ad with id '163981174974' was updated.
Ad with id '163981174986' was updated.
Ad with id '37710296863' was updated.
Ad with id '37710296983' was updated.
Ad with id '37710297103' was updated.
Ad with id '163981174968' was updated.
Ad with id '163981174971' was updated.
Ad with id '163981174977' was updated.
Ad with id '163981174980' was updated.
Ad with id '163981174983' was updated.
Ad with id '163981174989' was up

[]
[]


In [117]:
greater_than_3.groupby(['Ad group ID']).aggregate(lambda x: list(x))

,Campaign ID,Ad ID_x,Ad ID_y,Impressions,Clicks,Cost,ctr
Ad group ID,,,,,,,
45713167844,"[874677791, 874677791, 874677791, 874677791]","[4, 4, 4, 4]","[205016021996, 205016021999, 205016022125, 205...","[356, 379, 166, 337]","[110, 107, 0, 19]","[2650000, 2920000, 0, 560000]","[0.308988764045, 0.282321899736, 0.0, 0.056379..."
45713169524,"[874677791, 874677791, 874677791, 874677791]","[4, 4, 4, 4]","[205016022362, 205016022365, 205016022236, 205...","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]"


In [315]:
#greater_than_3.groupby(['Ad group ID']).apply(lambda tdf: pd.Series(dict([[vv,tdf[vv].unique().tolist()] for vv in tdf if vv not in ['A']])  )) 
greater_than_3.groupby(['Campaign ID', 'Ad group ID']).apply(lambda tdf: pd.Series(dict([[vv,tdf[vv].unique().tolist()] for vv in tdf if vv not in ['A']])  )) 

Ad ID_x  \
Campaign ID Ad group ID           
874677791   45713167844     [4]   
            45713169524     [4]   

                                                                   Ad ID_y  \
Campaign ID Ad group ID                                                      
874677791   45713167844  [205016021996, 205016021999, 205016022125, 205...   
            45713169524  [205016022362, 205016022365, 205016022236, 205...   

                           Ad group ID  Campaign ID             Clicks  \
Campaign ID Ad group ID                                                  
874677791   45713167844  [45713167844]  [874677791]  [110, 107, 0, 19]   
            45713169524  [45713169524]  [874677791]                [0]   

                                                  Cost           Impressions  \
Campaign ID Ad group ID                                                        
874677791   45713167844  [2650000, 2920000, 0, 560000]  [356, 379, 166, 337]   
            45713169524                            [0]                   [0]   

                                                                       ctr  
Campaign ID Ad group ID                                                     
874677791   45713167844  [0.308988764045, 0.282321899736, 0.0, 0.056379...  
            45713169524                                              [0.0]